In [1]:
import cv2
import dlib
import numpy as np
import os

In [2]:
face_detector = dlib.get_frontal_face_detector()

In [3]:
feature_predictor = dlib.shape_predictor('model_assets/shape_predictor_68_face_landmarks.dat')

In [4]:
capture = cv2.VideoCapture(2)

while True:
    _, frame = capture.read()
    
    grayscale = cv2.cvtColor(src=frame, code=cv2.COLOR_BGR2GRAY)
    
    faces = face_detector(grayscale)
    
    for face in faces:
        x1 = face.left()
        y1 = face.top()
        x2 = face.right()
        y2 = face.bottom()
        
        landmarks = feature_predictor(image = grayscale, box=face)
        
        for n in range(0,68):
            x = landmarks.part(n).x
            y = landmarks.part(n).y
            
            cv2.circle(img=frame, center=(x,y), radius=8, color= (0,255,0), thickness=-1)
    
    cv2.imshow(winname = "Facial Feature Detection", mat=frame)
        
    if cv2.waitKey(20) & 0xFF == 27:
        break
    
capture.release()
    
cv2.destroyAllWindows()


In [4]:
capture = cv2.VideoCapture(2)

while True:
    _, frame = capture.read()
    
    grayscale = cv2.cvtColor(src=frame, code=cv2.COLOR_BGR2GRAY)
    
    faces = face_detector(grayscale)
    
    for face in faces:
        x1 = face.left()
        y1 = face.top()
        x2 = face.right()
        y2 = face.bottom()
        
        landmarks = feature_predictor(image = grayscale, box=face)
        vector = np.empty([68, 2], dtype = 'float32')
        for b in range(68):
            vector[b][0] = landmarks.part(b).x
            vector[b][1] = landmarks.part(b).y

        dst_pts = np.array(
                [
                    vector[0],
                    vector[1],
                    vector[2],
                    vector[3],
                    vector[4],
                    vector[5],
                    vector[6],
                    vector[7],
                    vector[8],
                    vector[9],
                    vector[10],
                    vector[11],
                    vector[12],
                    vector[13],
                    vector[14],
                    vector[15],
                    vector[16],
                    vector[49],
                    vector[50],
                    vector[51],
                    vector[52],
                    vector[53],
                    vector[54],
                    vector[55],
                    vector[56],
                    vector[57],
                    vector[58],
                    vector[59],
                    vector[60],
                    vector[61],
                    vector[62],
                    vector[62],
                    vector[63],
                    vector[64],
                    vector[65],
                    vector[66],
                    vector[67],
                ])
        
        src_pts_1 =  np.genfromtxt('model_assets/kermit_labels.csv', delimiter=',')
        src_pts = np.empty([37,2], dtype = 'float32')
        for i, row in enumerate(src_pts_1):
            src_pts[i][0] = row[1]
            src_pts[i][1] = row[2]
            
        # load mask image
        mask_img = cv2.imread('model_assets/kermit.png', cv2.IMREAD_UNCHANGED)
        mask_img = mask_img.astype(np.float32)
        
        M, _ = cv2.findHomography(src_pts, dst_pts)
        
        transformed_mask = cv2.warpPerspective(
            mask_img,
            M,
            (frame.shape[1], frame.shape[0]),
            None,
            cv2.INTER_LINEAR,
            cv2.BORDER_CONSTANT,
        )
        
        # mask overlay
        alpha_mask = transformed_mask[:, :, 3]
        alpha_image = 1.0 - alpha_mask
        trans_h = transformed_mask.shape[0]
        trans_w = transformed_mask.shape[1]

        mask = (transformed_mask[:,:,3] != 0).flatten()
        kermit = transformed_mask[:,:,0:3].flatten().reshape(-1,3)
        cap = frame.flatten().reshape(-1,3)
        cap[mask] = kermit[mask]
        final = cap.reshape(frame.shape[0],frame.shape[1],3)

        cv2.imshow(winname = "Facial Feature Detection", mat=final)
        
    if cv2.waitKey(20) & 0xFF == 27:
        break

capture.release()
    
cv2.destroyAllWindows()

In [5]:
capture = cv2.VideoCapture(2)

while True:
    _, frame = capture.read()

    grayscale = cv2.cvtColor(src=frame, code=cv2.COLOR_BGR2GRAY)

    faces = face_detector(grayscale)

    for face in faces:
        x1 = face.left()
        y1 = face.top()
        x2 = face.right()
        y2 = face.bottom()

        landmarks = feature_predictor(image = grayscale, box=face)
        
        #create an empty numpy array and load it with the x and y coordinates of our predicted landmarks
        vector = np.empty([68, 2], dtype = 'float32')
        for b in range(68):
            vector[b][0] = landmarks.part(b).x
            vector[b][1] = landmarks.part(b).y
        
        #choose which landmarks we want to use with out overlay
        dst_pts = np.array(
                [
                    vector[0],
                    vector[1],
                    vector[2],
                    vector[3],
                    vector[4],
                    vector[5],
                    vector[6],
                    vector[7],
                    vector[8],
                    vector[9],
                    vector[10],
                    vector[11],
                    vector[12],
                    vector[13],
                    vector[14],
                    vector[15],
                    vector[16],
                    vector[49],
                    vector[50],
                    vector[51],
                    vector[52],
                    vector[53],
                    vector[54],
                    vector[55],
                    vector[56],
                    vector[57],
                    vector[58],
                    vector[59],
                    vector[60],
                    vector[61],
                    vector[62],
                    vector[62],
                    vector[63],
                    vector[64],
                    vector[65],
                    vector[66],
                    vector[67],
                ])
        
        #load the x and y coordinates from our annotation and load them into a numpy array as well.
        src_pts_1 =  np.genfromtxt('model_assets/gonzo_labels.csv', delimiter=',')
        src_pts = np.empty([37,2], dtype = 'float32')
        for i, row in enumerate(src_pts_1):
            src_pts[i][0] = row[1]
            src_pts[i][1] = row[2]

        # load overlay image
        mask_img = cv2.imread('model_assets/gonzo.png', cv2.IMREAD_UNCHANGED)
        mask_img = mask_img.astype(np.float32)

        #Create a transformation kernel to modify the overlay to the points
        M, _ = cv2.findHomography(src_pts, dst_pts)

        #pass the kernel over the overlay and resize to fit the frame
        transformed_mask = cv2.warpPerspective(
            mask_img,
            M,
            (frame.shape[1], frame.shape[0]),
            None,
            cv2.INTER_LINEAR,
            cv2.BORDER_CONSTANT,
        )

        #mask overlay
        alpha_mask = transformed_mask[:, :, 3]
        alpha_image = 1.0 - alpha_mask
        
        #Apply a mask of the overlay to the frame, then replace the mapped pixels with the pixels of our warped overlay
        mask = (transformed_mask[:,:,3] != 0).flatten()
        piggy = transformed_mask[:,:,0:3].flatten().reshape(-1,3)
        cap = frame.flatten().reshape(-1,3)
        cap[mask] = piggy[mask]
        final = cap.reshape(frame.shape[0],frame.shape[1],3)

        #Show the final product
        cv2.imshow(winname = "Facial Feature Detection", mat=final)
        
    if cv2.waitKey(20) & 0xFF == 27:
        break

capture.release()
    
cv2.destroyAllWindows()
